In [ ]:
import os
from copy import deepcopy
import jax
import sys
sys.path.append("..")

from federated_library.distributions import convert_to_federated_data
from federated_library.dataset_loader import load_tf_dataset
from federated_library.train_fed_avg import fed_avg_gridsearch
from constants import SKEWS, DATASETS, HP_GRID, NR_PARTIES

In [ ]:
print(jax.local_devices())

In [ ]:
def run(params, ds, test_split, ds_info, display):
    clients_set = params['clients_set']
    skews_set = params['skews_set']

    params = deepcopy(params)
    del params['clients_set']
    del params['skews_set']

    for nr_clients in clients_set:
        ds_info['num_clients'] = nr_clients
        for skew in skews_set:
            print(f"{nr_clients} clients, skew={skew}")
            params['skew'] = [skew]
            fedavg_hyperparams_grid_search_res = fed_avg_gridsearch(
                params, ds, test_split, ds_info, display=display
            )

            sorted_res = sorted(fedavg_hyperparams_grid_search_res,
                                key=lambda e: e[0], reverse=True)

            if not os.path.exists("results/"):
                os.makedirs("results/")

            file_name = f"results/{dataset_name}_{skew_type}_skew_{skew}_{nr_clients}clients.txt"
            print(file_name)
            textfile = open(file_name, "w")
            for line in sorted_res:
                textfile.write(str(line) + "\n")
            textfile.close()

In [ ]:
for dataset_name in DATASETS:
    for skew_type in SKEWS.keys():
        print(f"DATASET: {dataset_name}, SKEW_TYPE: {skew_type}")
        ds, (x_test, y_test), ds_info = load_tf_dataset(
            dataset_name=dataset_name, skew_type=skew_type,
            decentralized=False, display=False
        )

        test_split = convert_to_federated_data(x_test, y_test, ds_info,
                                               is_train=False)

        hp_configs = dict(
            act_fn=[jax.nn.relu],
            client_lr=[0.01],
            server_lr=HP_GRID["lr"],
            client_momentum=[0.01],
            server_momentum=HP_GRID["mom"],
            batch_size=HP_GRID["bs"],
            epochs_per_round=[2],
            rounds=[30],
            runs=[1],
            clients_set=NR_PARTIES,
            skews_set=SKEWS[skew_type]
        )

        run(hp_configs, ds, test_split, ds_info, display=True)